# Milestone 1 - EDA and Preprocessing data 

- Load dataset
- Explore the dataset and ask atleast 5 questions to give you a better understanding of the data provided to you. 
- Visualise the answer to these 5 questions.
- Cleaing the data
- Observe missing data and comment on why you believe it is missing(MCAR,MAR or MNAR) 
- Observe duplicate data
- Observe outliers
- After observing outliers,missing data and duplicates, handle any unclean data.
- With every change you are making to the data you need to comment on why you used this technique and how has it affected the data(by both showing the change in the data i.e change in number of rows/columns,change in distrubution, etc and commenting on it).
- Data transformation and feature engineering
- Add a new column named 'Week number' and discretisize the data into weeks according to the dates.Tip: Change the datatype of the date feature to datetime type instead of object.
- Encode any categorical feature(s) and comment on why you used this technique and how the data has changed.
- Identify feature(s) which need normalisation and show your reasoning.Then choose a technique to normalise the feature(s) and comment on why you chose this technique.
- Add atleast two more columns which adds more info to the dataset by evaluating specific feature(s). I.E( Column indicating whether the accident was on a weekend or not). 
- For any imputation with arbitrary values or encoding done, you have to store what the value imputed or encoded represents in a new csv file. I.e if you impute a missing value with -1 or 100 you must have a csv file illustrating what -1 and 100 means. Or for instance, if you encode cities with 1,2,3,4,etc what each number represents must be shown in the new csv file.
- Load the new dataset into a csv file.
- **Extremely Important note** - Your code should be as generic as possible and not hard-coded and be able to work with various datasets. Any hard-coded solutions will be severely penalised.
- Bonus: Load the dataset as a parquet file instead of a csv file(Parquet file is a compressed file format).

# 1 - Extraction

In [38]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np



In [46]:
pd.set_option('display.max_columns', None)
dataset = 'dataset/1980_Accidents_UK.csv'
df_accidents_1980 = pd.read_csv(dataset, index_col=None)
df_accidents_1980.head()


/tmp/ipykernel_9355/3239739217.py:3: DtypeWarning: Columns (14,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accidents_1980 = pd.read_csv(dataset, index_col=None)


,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,19800113MKA18,1980,0113MKA18,288680.0,118000.0,NaN,NaN,Metropolitan Police,Slight,1,1,03/09/1980,Wednesday,12:50,Westminster,-1,-1,A,40.0,Single carriageway,30.0,Crossroads,Auto traffic signal,-1,NaN,Data missing or out of range,Data missing or out of range,Daylight,Other,Dry,Data missing or out of range,None,Data missing or out of range,Data missing or out of range,Data missing or out of range,-1
1,19800113MKB18,1980,0113MKB18,280670.0,978000.0,NaN,NaN,Metropolitan Police,Slight,3,1,03/09/1980,Wednesday,12:50,Westminster,-1,-1,A,4.0,Dual carriageway,30.0,Not at junction or within 20 metres,Data missing or out of range,-1,NaN,Data missing or out of range,Data missing or out of range,Daylight,Other,Dry,Data missing or out of range,None,Data missing or out of range,Data missing or out of range,Data missing or out of range,-1
2,19800114MAA22,1980,0114MAA22,266880.0,562000.0,NaN,NaN,Metropolitan Police,Slight,2,2,04/09/1980,Thursday,12:00,Camden,-1,-1,A,502.0,NaN,30.0,T or staggered junction,Give way or uncontrolled,-1,NaN,Data missing or out of range,Data missing or out of range,Daylight,Other,Dry,Data missing or out of range,None,Data missing or out of range,Data missing or out of range,Data missing or out of range,-1
3,19800114MAB22,1980,0114MAB22,290780.0,442000.0,NaN,NaN,Metropolitan Police,Slight,1,1,04/09/1980,Thursday,12:00,Camden,-1,-1,A,5202.0,Single carriageway,30.0,T or staggered junction,Give way or uncontrolled,-1,NaN,None within 50 metres,Zebra,Daylight,Other,Dry,Data missing or out of range,None,Data missing or out of range,Data missing or out of range,Data missing or out of range,-1
4,19800116UGA99,1980,0116UGA99,307670.0,459000.0,NaN,NaN,Metropolitan Police,Serious,1,1,06/09/1980,Saturday,20:30,Lambeth,-1,-1,A,23.0,NaN,30.0,Crossroads,Auto traffic signal,-1,NaN,Data missing or out of range,Data missing or out of range,Darkness - lights lit,Other,Dry,Data missing or out of range,None,Data missing or out of range,Data missing or out of range,Data missing or out of range,-1


# 2- EDA

**Does accident_index have uniqe values and what is it's relatiion to accident_year and accident_refrence?**

In [57]:
accident_index_unique_counts = len(df_accidents_1980["accident_index"].unique())
accident_refernce_unique_counts = len(df_accidents_1980["accident_reference"].unique())
number_of_entries = len(df_accidents_1980)
"Number of Unique accident_index: " + \
    str(accident_index_unique_counts), "Number of Unique accident_refrence: " + str(accident_refernce_unique_counts), "Number of total entries:  " + str(number_of_entries)


('Number of Unique accident_index: 250958',
 'Number of Unique accident_refrence: 250958',
 'Number of total entries:  250958')

In [66]:
(np.equal((df_accidents_1980["accident_year"].astype(str) +
          df_accidents_1980["accident_reference"]), df_accidents_1980["accident_index"])).all()

df_accidents_1980["accident_year"].unique()

array([1980])

# 3 - Cleaning Data

## Observing Missing and duplicate Data

## Handling Missing data

## Findings and conclusions

## Observing outliers

## Handling outliers

## Findings and conclusions

# 4 - Data transformation

## 4.1 - Discretization

## 4.11 - Findings and conclusions

## 4.2 - Encoding

## 4.22 - Findings and conlcusions

## 4.3 - Normalisation 

## 4.31 - Findings and conclusions

## 4.4 - Adding more columns

## 4.41 - Findings and concluisons

## 4.5 - Csv file for lookup

## 5- Exporting the dataframe to a csv file or parquet